In [1]:
from memory_reveal_cards_game import MemoryGame
from minimum_vertex_cover_vertices import MinimumVertexCoverVertices
from domino import Domino, bigger, random_player, hardest, random_players, random_hands, hardest_bigger
from logChoice import LogChoice
from bigChoice import BigChoice
from fixedParameters import FixedParameters
from pso import pso_solve
import numpy as np
import random
from utils import smallest_dominating_set, greedy_dominating_set, generate_random_graph
from metaheuristicParameters import PSOParameters
from transformerParameters import TransformerParametersSVD, TransformerParametersSequential

In [3]:
choice = LogChoice()
choiceParameters = FixedParameters()
problem = MemoryGame(choice, choiceParameters, 0.01, 36, 50)

In [4]:
init_positions = lambda num_particles: [np.random.random((1, 2)) * 2 - 1 for i in range(num_particles)]
init_velocities = lambda num_particles: [np.zeros((1, 2)) for i in range(num_particles)]   

def fitness(p):
    total = 0
    count = 10
    ol = 0

    choiceParameters.set(p[0][0], p[0][1])

    for i in range(count):
        cards = problem.random_cards()
        v = problem.run(cards)

        if v == problem.limit:
            ol += 1
        else:
            total += v

    if count > ol:
        return ol + 2 * (total / (count - ol)) / problem.limit
    else:
        return ol + 2

p, score, positions, velocities = pso_solve(20, 10
                                            , init_positions
                                            , init_velocities
                                            , fitness
                                            , lambda p: choice.apply_constraints(p[0][0], p[0][1])
                                            , lambda score: score < 0.8
                                            , False
                                            )

print(p)
print(score)

d:\SVD\tesis\logChoice.py:11: RuntimeWarning: overflow encountered in scalar power
  exp = x ** (1 / b)


[[ 0.17213467 -0.18865357]]
1.7919999999999998


In [13]:
def test(choice, p):
    choiceParameters = FixedParameters()
    problem = MemoryGame(choice, choiceParameters, 0.03, 36, 1000)
    total = 0
    count = 100
    ol = 0
    choiceParameters.set(p[0][0], p[0][1])

    for i in range(count):
        cards = problem.random_cards()
        v = problem.run(cards)

        if v == problem.limit:
            ol += 1
        else:
            total += v

    if count > ol:
        print(total / (count - ol), ol)
    else:
        print(ol)

In [14]:
test(LogChoice(), p)

87.14 0


In [21]:
test(BigChoice(), p)

29.858823529411765 15


In [ ]:
choice = LogChoice()
choiceParameters = FixedParameters()
problem = MinimumVertexCoverVertices(choice, choiceParameters)

In [25]:
dataset = []

In [26]:
for i in range(300):
    n = random.randint(1, 100)
    graph = generate_random_graph(n)
    m = smallest_dominating_set(graph) 
    n = greedy_dominating_set(graph)

    if n > m:
        dataset.append((graph, n))

print(len(dataset))

78


In [27]:
init_positions = lambda num_particles: [np.random.random((1, 2)) * 2 - 1 for i in range(num_particles)]
init_velocities = lambda num_particles: [np.zeros((1, 2)) for i in range(num_particles)]   

def fitness(p):
    losses = 0
    choiceParameters.set(p[0][0], p[0][1])

    for graph, greedy in dataset:
        if problem.run(graph) >= greedy:
            losses += 1

    return losses

p, score, positions, velocities = pso_solve(20, 10
                                            , init_positions
                                            , init_velocities
                                            , fitness
                                            , lambda p: choice.apply_constraints(p[0][0], p[0][1])
                                            , lambda score: score == 0
                                            , False
                                            )

print(p)
print(score)

[[ 1.00000000e-11 -5.81720977e-01]]
63


In [28]:
count = 200
total = 0

choiceParameters.set(p[0][0], p[0][1])

for j in range(count):
    win_C = 0

    for i in range(count):
        n = random.randint(1, 200)
        graph = generate_random_graph(n)
        m = problem.run(graph) 
        n = greedy_dominating_set(graph)

        if min(m, n) < n:
            win_C += 1

    total += win_C 

print(f"won: {total/count}")

won: 8.355


In [13]:
choice = LogChoice()

choiceParameters = FixedParameters()
problem = Domino(choice, choiceParameters)

psoParameters = PSOParameters()
problemFit = Domino(choice, psoParameters)

In [50]:
count_fitness = 100

def fitness(p):
    losses = 0

    choiceParameters.set(p[0][0], p[0][1])

    for i in range(count_fitness):
        players, main_player = random_players([bigger, hardest, random_player], 2, hardest_bigger)
        hands = random_hands(len(players))
        ws = problem.run(hands, players, main_player)

        if isinstance(ws, int) and ws != main_player:
            losses += 1
        elif isinstance(ws, list) and main_player not in ws:
            losses += 1

    return losses

score, p = psoParameters.fit(fitness, choice.apply_constraints, 2, 10, 25)
print(score, p)

24 [ 1.00000000e-11 -7.79639042e-01]


In [60]:
count = 200
losses = 0

for i in range(count):
    players, main_player = random_players([bigger, hardest, random_player], 2, hardest_bigger)
    hands = random_hands(len(players))
    ws = problemFit.run(hands, players, main_player)

    if isinstance(ws, int) and ws != main_player:
        losses += 1
    elif isinstance(ws, list) and main_player not in ws:
        losses += 1

print(losses)

79


In [3]:
def get_sequences_from_start(list):
    return [list[:pos + 1] for pos in range(len(list))]

In [5]:
X, Y = [], []

for c in range(20):
    print(c)
    players, main_player = random_players([bigger, hardest, random_player], 4, hardest_bigger)
    hands = random_hands(len(players))

    count_fitness = 20

    def fitness(p):
        losses = 0

        choiceParameters.set(p[0][0], p[0][1])

        for i in range(count_fitness):
            ch = [[f for f in hand] for hand in hands]
            ws = problem.run(ch, players, main_player)

            if isinstance(ws, int) and ws != main_player:
                losses += 1
            elif isinstance(ws, list) and main_player not in ws:
                losses += 1

        return losses

    score, p = psoParameters.fit(fitness, choice.apply_constraints, 2, 10, 25)

    count = 500
    losses = 0

    for i in range(count):
        ch = [[f for f in hand] for hand in hands]
        ws = problemFit.run(ch, players, main_player)

        if isinstance(ws, int) and ws != main_player:
            losses += 1
        elif isinstance(ws, list) and main_player not in ws:
            losses += 1

    if (losses / count) > (score / 20) + 0.1:
        count_fitness = 200
        score, p = psoParameters.fit(fitness, choice.apply_constraints, 2, 10, 25)

    ch = [[f for f in hand] for hand in hands]
    problemFit.run(ch, players, main_player)

    X.extend(get_sequences_from_start(problemFit.embeddings))
    Y.extend([p] * len(problemFit.embeddings))

0


d:\SVD\tesis\Decision-Making-Optimization\logChoice.py:16: RuntimeWarning: overflow encountered in double_scalars
  exp = a ** (1 / b)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [3]:
from pickle import load, dump

with open('xy_domino', 'rb') as f:
    X, Y = load(f)

In [4]:
print(len(X), len(Y))

169 169


In [7]:
transformerParameters = TransformerParametersSVD()

In [12]:
#for _ in range(50):
preds, error = transformerParameters.train(X[:10], Y[:10])
print(error)

with open('tp_domino', 'wb') as f:
    dump(transformerParameters, f)

Epoch 1/10, Loss: 0.0171
Epoch 2/10, Loss: 0.4970
Epoch 3/10, Loss: 0.1562
Epoch 4/10, Loss: 0.0531
Epoch 5/10, Loss: 0.1295
Epoch 6/10, Loss: 0.1877
Epoch 7/10, Loss: 0.1083
Epoch 8/10, Loss: 0.0214
Epoch 9/10, Loss: 0.0060
Epoch 10/10, Loss: 0.0394
MSE: 0.0671
0.067133956


In [87]:
problemTrans = Domino(choice, psoParameters)

count = 200
losses = 0

for i in range(count):
    players, main_player = random_players([bigger, hardest, random_player], 4, hardest_bigger)
    hands = random_hands(len(players))
    ws = problemTrans.run(hands, players, main_player)

    if isinstance(ws, int) and ws != main_player:
        losses += 1
    elif isinstance(ws, list) and main_player not in ws:
        losses += 1

print(losses)

133
